# Markov Transition Matrix between Delinquency Statuses

This technical report is oriented to calculate the probability matrix in the "actual" dataset to see the next month distribution. The following transition matrices were taken in consideration: 

- Transition Matrix for all loans in a single markov chain in the full dataset.
- Transition Matrix for a markov chain per loan in the full dataset.
- Transition Matrix per Loan without consecutive Paid-Off status' in the full dataset.
- Transition Matrix per Loan without invalid transitions and consecutive Paid-Off status for Subprime, Prime and full dataset.
- Transition Matrix for unknown type of loans. 

This is an investigation based on the actual data, not related to the training model shown before. There are some open questions for further discussion.

In [7]:
import sys
import os
import pandas as pd
from pathlib import Path
import numpy as np
import datetime


nb_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'data')
pdata_dir = os.path.join(Path(os.getcwd()).parents[0], 'data', 'processed')
if nb_dir not in sys.path:
    sys.path.insert(0, nb_dir)
# print(sys.path)
import make_dataset as md
import build_data as bd
import get_raw_data as grd
import features_selection as fs

models_dir = os.path.join(Path(os.getcwd()).parents[0], 'src', 'models')
if models_dir not in sys.path:
    sys.path.insert(0, models_dir)
import nn_real as nn

## Original Feeding Set

In [5]:
all_data = grd.read_df(45)
print(all_data.shape)
pd.options.display.max_columns = 1500
all_data.head(15)

2018-04-03 14:52:15,343 - read_df - INFO - merged shape:(4055501, 195)
2018-04-03 14:52:34,495 - read_df - INFO - dynamic shape:(4056947, 34) static shape:(67637, 164) unemployment shape: (17212, 3) merged shape: (4054864, 196)


(4054864, 196)


,LOAN_ID,ASOFMONTH,MBA_DAYS_DELINQUENT,CURRENT_INTEREST_RATE,LLMA2_CURRENT_INTEREST_SPREAD,LOANAGE,CURRENT_BALANCE,SCHEDULED_PRINCIPAL,SCHEDULED_MONTHLY_PANDI,LLMA2_HIST_LAST_12_MONTHS_MIS,LLMA2_C_IN_LAST_12_MONTHS,LLMA2_30_IN_LAST_12_MONTHS,LLMA2_60_IN_LAST_12_MONTHS,LLMA2_90_IN_LAST_12_MONTHS,LLMA2_FC_IN_LAST_12_MONTHS,LLMA2_REO_IN_LAST_12_MONTHS,YEAR,MONTH,TIME,CURRENT_INTEREST_RATE_nan,MBA_DAYS_DELINQUENT_nan,SCHEDULED_MONTHLY_PANDI_nan,SCHEDULED_PRINCIPAL_nan,MBA_DELINQUENCY_STATUS_next,MBA_DELINQUENCY_STATUS_0,MBA_DELINQUENCY_STATUS_3,MBA_DELINQUENCY_STATUS_6,MBA_DELINQUENCY_STATUS_9,MBA_DELINQUENCY_STATUS_C,MBA_DELINQUENCY_STATUS_F,MBA_DELINQUENCY_STATUS_R,CURRENT_INVESTOR_CODE_253,MBA_DELINQUENCY_STATUS,FICO_SCORE_ORIGINATION,INITIAL_INTEREST_RATE,ORIGINAL_LTV,ORIGINAL_BALANCE,BACKEND_RATIO,SALE_PRICE,NUMBER_OF_UNITS,LLMA2_APPVAL_LT_SALEPRICE,LLMA2_ORIG_RATE_SPREAD,MARGIN,PERIODIC_RATE_CAP,PERIODIC_RATE_FLOOR,LIFETIME_RATE_CAP,LIFETIME_RATE_FLOOR,RATE_RESET_FREQUENCY,PAY_RESET_FREQUENCY,FIRST_RATE_RESET_PERIOD,LLMA2_PRIME,LLMA2_SUBPRIME,BACKEND_RATIO_nan,FIRST_RATE_RESET_PERIOD_nan,LIFETIME_RATE_CAP_nan,LIFETIME_RATE_FLOOR_nan,MARGIN_nan,PAY_RESET_FREQUENCY_nan,PERIODIC_RATE_CAP_nan,PERIODIC_RATE_FLOOR_nan,RATE_RESET_FREQUENCY_nan,SALE_PRICE_nan,ORIGINAL_TERM_180,ORIGINAL_TERM_240,ORIGINAL_TERM_360,LLMA2_VINTAGE_2005,BUYDOWN_FLAG_N,BUYDOWN_FLAG_U,BUYDOWN_FLAG_Y,NEGATIVE_AMORTIZATION_FLAG_N,NEGATIVE_AMORTIZATION_FLAG_U,NEGATIVE_AMORTIZATION_FLAG_Y,PREPAY_PENALTY_FLAG_N,PREPAY_PENALTY_FLAG_U,PREPAY_PENALTY_FLAG_Y,OCCUPANCY_TYPE_1,OCCUPANCY_TYPE_2,OCCUPANCY_TYPE_3,OCCUPANCY_TYPE_U,PRODUCT_TYPE_10,PRODUCT_TYPE_20,PRODUCT_TYPE_30,PRODUCT_TYPE_50,PRODUCT_TYPE_51,PRODUCT_TYPE_52,PRODUCT_TYPE_54,PRODUCT_TYPE_5A,PRODUCT_TYPE_5Z,PRODUCT_TYPE_60,PRODUCT_TYPE_63,PRODUCT_TYPE_70,PRODUCT_TYPE_80,PRODUCT_TYPE_81,PRODUCT_TYPE_82,PRODUCT_TYPE_83,PRODUCT_TYPE_84,PRODUCT_TYPE_8Z,PRODUCT_TYPE_U,LOAN_PURPOSE_1,LOAN_PURPOSE_2,LOAN_PURPOSE_3,LOAN_PURPOSE_5,LOAN_PURPOSE_6,LOAN_PURPOSE_7,LOAN_PURPOSE_8,LOAN_PURPOSE_9,LOAN_PURPOSE_B,LOAN_PURPOSE_U,DOCUMENTATION_TYPE_1,DOCUMENTATION_TYPE_2,DOCUMENTATION_TYPE_3,DOCUMENTATION_TYPE_U,CHANNEL_1,CHANNEL_2,CHANNEL_3,CHANNEL_4,CHANNEL_7,CHANNEL_8,CHANNEL_9,CHANNEL_A,CHANNEL_C,CHANNEL_D,CHANNEL_U,LOAN_TYPE_1,LOAN_TYPE_2,LOAN_TYPE_3,LOAN_TYPE_4,LOAN_TYPE_U,IO_FLAG_N,IO_FLAG_U,IO_FLAG_Y,CONVERTIBLE_FLAG_N,CONVERTIBLE_FLAG_U,CONVERTIBLE_FLAG_Y,POOL_INSURANCE_FLAG_N,POOL_INSURANCE_FLAG_U,POOL_INSURANCE_FLAG_Y,STATE_AE,STATE_AK,STATE_AL,STATE_AP,STATE_AR,STATE_AZ,STATE_CA,STATE_CO,STATE_CT,STATE_DC,STATE_DE,STATE_FL,STATE_GA,STATE_GU,STATE_HI,STATE_IA,STATE_ID,STATE_IL,STATE_IN,STATE_KS,STATE_KY,STATE_LA,STATE_MA,STATE_MD,STATE_ME,STATE_MI,STATE_MN,STATE_MO,STATE_MS,STATE_MT,STATE_NC,STATE_ND,STATE_NE,STATE_NH,STATE_NJ,STATE_NM,STATE_NN,STATE_NV,STATE_NY,STATE_OH,STATE_OK,STATE_OR,STATE_PA,STATE_PR,STATE_RI,STATE_SC,STATE_SD,STATE_TN,STATE_TX,STATE_UT,STATE_VA,STATE_VI,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY,STATE,UR
0,1152309363,2005-02-01,0,5.375,-0.255,1,179000.000000,0.000000,1002.349976,0,1,0,0,0,0,0,2005,2,421,0,0,0,0,C,0,0,0,0,1,0,0,1,C,752.0,5.375,76.169998,179000.0,33.369999,0.0,1,0.0,-0.335,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,MN,4.2
1,1152317734,2005-02-01,0,5.375,-0.255,1,150300.000000,0.000000,673.219971,0,1,0,0,0,0,0,2005,2,421,0,0,0,0,C,0,0,0,0,1,0,0,1,C,706.0,5.375,79.940002,150300.0,24.030001,0.0,1,0.0,-0.335,0.0,2.0,3.375,6.0,0.0,12.0,0.0,36.0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Ordering the Original Dataset
Ordering the dataset by ascending 'LOAN_ID' and 'ASOFMONTH' to count the transition states adequately.

In [8]:
all_data = all_data.sort_values(['LOAN_ID', 'ASOFMONTH'], ascending=[1, 1])
all_data.head(15)
all_data[['LOAN_ID', 'ASOFMONTH', 'MBA_DELINQUENCY_STATUS_next']]

,LOAN_ID,ASOFMONTH,MBA_DELINQUENCY_STATUS_next
3548663,131175200,2005-10-01,3
3549190,131175200,2005-11-01,6
3548834,131175200,2005-12-01,F
3549381,131175200,2006-01-01,F
3549637,131175200,2006-02-01,9
3550739,131175200,2006-03-01,9
3549904,131175200,2006-04-01,0
1657429,217010733,2011-03-01,C
1657755,217010733,2011-04-01,C
1658074,217010733,2011-05-01,C


## Extracting the Original labels

In [9]:
labels = bd.extract_numeric_labels(all_data, label_column='MBA_DELINQUENCY_STATUS_next')

2018-04-03 14:58:18,645 - extract_numeric_labels - INFO - mapped labels: {'0': 0, '3': 1, '6': 2, '9': 3, 'C': 4, 'F': 5, 'R': 6}
2018-04-03 14:58:21,991 - extract_numeric_labels - INFO - ...Labels extracted from Dataset...
2018-04-03 14:58:21,992 - extract_numeric_labels - INFO - Size of the dataset after extract_labels:(4054864, 195)


In [36]:
display(labels)

3548663    1
3549190    2
3548834    5
3549381    5
3549637    3
3550739    3
3549904    0
1657429    4
1657755    4
1658074    4
1658388    4
1659817    4
1660108    4
1658675    4
1658958    4
1659235    4
1659507    4
1661680    4
1660904    4
1660364    4
1660631    4
1661164    4
1661408    4
1661906    4
1662143    4
1662375    4
1662605    4
1662833    4
1663079    4
1663322    4
          ..
3834281    4
3833750    4
3834011    4
3834542    4
3834802    4
3149484    1
3149813    1
3150138    1
3151014    1
3150421    1
3150692    1
3151534    1
3151242    1
3152041    2
3151779    2
3152549    2
3153054    3
3152290    1
3152800    1
3153550    1
3153295    1
3153797    1
3154037    4
3154277    4
3154484    4
3155089    4
3154688    4
3155289    4
3154886    4
3155487    4
Name: MBA_DELINQUENCY_STATUS_next, Length: 4054864, dtype: int64

## Transitions between statuses
From Paper (Sirignano et. al, 2017): 

"A mortgage is determined to be 1 month delinquent if no payment has been made by the last day of the month 
and the payment was due on the ﬁrst day of the month." 

"The transition matrices highlight that mortgages frequently transition back and forth between current and any other delinquency states. Disruptions in cashﬂow to the lender or servicer are common due to the mortgage being behind payment. Similarly, even loans that are extremely delinquent may return to current; the transition from foreclosed back to current is actually a relatively frequent occurrence. A foreclosure could get cured via paying the outstanding balance, there could be a pre-auction sale that covers all or some of the amount outstanding, or there could be a sale at the foreclosure auction that covers all or some of the amount outstanding. Any of these will register as a foreclosure to paid oﬀ transition. Mortgages can also transition directly from current, 30 days delinquent, 60 days delinquent, or 90+ days delinquent to REO via a 'deed in lieu of foreclosure' ".

The transition matrix calculation depends on not only the next status but also the next month. If certain records for a loan are not consecutive in months, these transitions will not be considered in the calculation.

In [11]:
def transition_matrix(M, transitions):
    transition_chain = zip(transitions,transitions[1:])
    # print('size: ', len(list(transition_chain)))
    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1      

def probabilities_matrix(M):    
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]    

## Transition Matrix for all loans in a single markov chain
This function considers all original labels in a single markov chain and then calculates the transitions between statuses.

In [12]:
def markov_tmatrix_all_loans(labels):
    n = 1+ max(labels) #number of statuses
    M = [[0]*n for _ in range(n)]
    transition_matrix(M, labels)
    return M

## Transition Matrix - all Loans
The rows are the current status and the columns is the next month status according the dictionary: {'0': 0, '3': 1, '6': 2, '9': 3, 'C': 4, 'F': 5, 'R': 6}.

In [35]:
M = markov_tmatrix_all_loans(labels)
display('Transition Matrix - all Loans')
display(M)

'Transition Matrix - all Loans'

[[225949, 556, 70, 104, 37802, 122, 36],
 [659, 33541, 16247, 129, 27099, 24, 6],
 [169, 3752, 9317, 10453, 2901, 806, 2],
 [1395, 802, 1416, 51138, 3405, 8129, 336],
 [34663, 38906, 282, 202, 3466212, 142, 45],
 [1384, 126, 67, 4114, 1844, 58516, 1779],
 [420, 21, 1, 481, 1190, 91, 8012]]

In [49]:
display('Probabilities Matrix - all Loans')
probabilities_matrix(M)
for row in M: print(' '.join('{0:.3f}'.format(x) for x in row))

'Probabilities Matrix - all Loans'

0.854 0.002 0.000 0.000 0.143 0.000 0.000
0.008 0.432 0.209 0.002 0.349 0.000 0.000
0.006 0.137 0.340 0.381 0.106 0.029 0.000
0.021 0.012 0.021 0.768 0.051 0.122 0.005
0.010 0.011 0.000 0.000 0.979 0.000 0.000
0.020 0.002 0.001 0.061 0.027 0.863 0.026
0.041 0.002 0.000 0.047 0.116 0.009 0.784


## Transition Matrix for a markov chain per loan
Taking into consideration all the records grouped by loan, so it does not consider spurious states: false states between different loans and loans with a single record.

In [13]:
def markov_tmatrix_per_loan(data, labels):
    n = 1+ max(labels) # number of states
    M = [[0]*n for _ in range(n)]
    
    loan_id = data.groupby('LOAN_ID')['LOAN_ID'].count()    
    i =0
    for lid, count in loan_id.items():        
        labels_id = labels[i:i+count] # to separate by rows belonging to the same loan.                
        transition_matrix(M, labels_id)
        i += count    
    return M

In [53]:
M = markov_tmatrix_per_loan(all_data, labels)
display ('Transition Matrix per Loan')
display(M)

'Transition Matrix per Loan'

[[225070, 1, 0, 10, 8, 13, 6],
 [641, 33522, 16242, 121, 26073, 14, 2],
 [164, 3745, 9317, 10453, 2559, 802, 1],
 [1381, 788, 1415, 51128, 2705, 8122, 336],
 [34388, 38554, 214, 66, 3443542, 21, 8],
 [1373, 112, 64, 4103, 896, 58503, 1777],
 [413, 3, 0, 474, 11, 80, 7993]]

In [54]:
display('Probabilities Matrix per Loan')
probabilities_matrix(M)
for row in M: print(' '.join('{0:.3f}'.format(x) for x in row))  

'Probabilities Matrix per Loan'

1.000 0.000 0.000 0.000 0.000 0.000 0.000
0.008 0.438 0.212 0.002 0.340 0.000 0.000
0.006 0.138 0.345 0.387 0.095 0.030 0.000
0.021 0.012 0.021 0.776 0.041 0.123 0.005
0.010 0.011 0.000 0.000 0.979 0.000 0.000
0.021 0.002 0.001 0.061 0.013 0.875 0.027
0.046 0.000 0.000 0.053 0.001 0.009 0.891


These matrices per loan are more realistic than the previous calculated matrices. However, there are inusual records when the current status is 0-'Paid-Off' and the next status is 0-'Paid-Off' as well. The same occurs in other states, it seems the dataset exhibts "invalid transitions".

## Delinquency statuses per Loan
To see how many records there are by loan, the full dataset was grouped by 'LOAN_ID', 'MBA_DELINQUENCY_STATUS_next'.

In [38]:
all_data['MBA_DELINQUENCY_STATUS_next'] = labels
delinquencies = all_data[['LOAN_ID', 'MBA_DELINQUENCY_STATUS_next', 'ASOFMONTH']].groupby(['LOAN_ID', 'MBA_DELINQUENCY_STATUS_next'], as_index=False).count().sort_values(by=['LOAN_ID', 'MBA_DELINQUENCY_STATUS_next', 'ASOFMONTH'])
display(delinquencies)

,LOAN_ID,MBA_DELINQUENCY_STATUS_next,ASOFMONTH
0,131175200,0,1
1,131175200,1,1
2,131175200,2,1
3,131175200,3,2
4,131175200,5,2
5,217010733,4,45
6,217166158,0,60
7,217236823,4,3
8,217237944,4,2
9,217238108,4,3


Number of Loans per Delinquency Status.

In [57]:
delinquencies[['MBA_DELINQUENCY_STATUS_next', 'ASOFMONTH']].groupby('MBA_DELINQUENCY_STATUS_next').count()

,ASOFMONTH
MBA_DELINQUENCY_STATUS_next,
0,39546
1,16400
2,9573
3,8350
4,65753
5,6499
6,2184


Number of Loan Monthly Payments per Delinquency Status.

In [58]:
delinquencies[['MBA_DELINQUENCY_STATUS_next', 'ASOFMONTH']].groupby('MBA_DELINQUENCY_STATUS_next').sum()

,ASOFMONTH
MBA_DELINQUENCY_STATUS_next,
0,264639
1,77705
2,27400
3,66621
4,3540453
5,67830
6,10216


## Delinquency 0 (Paid-Off)
There are loans with several records that have delinquency statuses in 0-'Paid Off' at the end of its Markov Chain. 
It is interesting to see the number of consecutive records in 0-'Paid Off' status inside each Loan Markov chain.

##### Number of 0-'Pay-Off' Delinquencies per Loan

In [59]:
delinquency_0 = delinquencies[delinquencies.MBA_DELINQUENCY_STATUS_next==0].sort_values(by=['ASOFMONTH', 'LOAN_ID'])
display (delinquency_0)

,LOAN_ID,MBA_DELINQUENCY_STATUS_next,ASOFMONTH
0,131175200,0,1
36,218036307,0,1
107,218047831,0,1
228,218087675,0,1
232,218088688,0,1
265,218519514,0,1
295,218845444,0,1
382,219145330,0,1
442,219412986,0,1
444,219413979,0,1


##### Seeing all loans with 0-'Paid-Off' delinquencies  above 3 (Sample)

In [61]:
d = delinquencies[(delinquencies['ASOFMONTH'] > 3) & (delinquencies['MBA_DELINQUENCY_STATUS_next']==0)]
data_0 = all_data[all_data['LOAN_ID'].isin(d.LOAN_ID)][['LOAN_ID', 'ASOFMONTH', 'MBA_DELINQUENCY_STATUS_next']]
display(data_0)

,LOAN_ID,ASOFMONTH,MBA_DELINQUENCY_STATUS_next
2339640,217166158,2009-12-01,0
2343048,217166158,2010-01-01,0
2347941,217166158,2010-02-01,0
2344680,217166158,2010-03-01,0
2346282,217166158,2010-04-01,0
2351332,217166158,2010-05-01,0
2349733,217166158,2010-06-01,0
2359832,217166158,2010-07-01,0
2353088,217166158,2010-08-01,0
2361551,217166158,2010-09-01,0


## Function dropping Invalid Delinquencies

This function delete rows in the full dataset that satisfy certain conditions. For example, in case of 0-'Paid-Off', these consecutive states could be considered as invalid when the diference between the months each other is less or equal to 31 days.

In [15]:
def invalid_consecutive_delinquencies (group):
    uw = []
    for i in range(0, (len(group)-1)):        
        if (group.iloc[i+1]['ASOFMONTH'] - group.iloc[i]['ASOFMONTH']) <= datetime.timedelta(days=31):             
            uw.append(i+1)
    
    return group.index[uw]


def drop_invalid_delinquency_status(data, function, status=None):
    # groups = all_data[['LOAN_ID', 'MBA_DELINQUENCY_STATUS_next', 'ASOFMONTH']][all_data['MBA_DELINQUENCY_STATUS_next']==0].groupby(['LOAN_ID', 'MBA_DELINQUENCY_STATUS_next'])
    if status==None:
        groups = data[['LOAN_ID', 'ASOFMONTH', 'MBA_DELINQUENCY_STATUS_next']].groupby(['LOAN_ID'])
    else:
        groups = data[['LOAN_ID', 'ASOFMONTH']][data['MBA_DELINQUENCY_STATUS_next']==status].groupby(['LOAN_ID'])
    iuw = pd.Index([]) # inmutable data structure
    for name, group in groups: # group is a DataFrame
        # print(name) # the whole composed index column: ('LOAN_ID', 'MBA_DELINQUENCY_STATUS_next')
        group = group.sort_values(by=['ASOFMONTH'], ascending=[1])            
        iuw = iuw.union(function(group))        
        
    if iuw!=[]:                        
        data.drop(iuw, inplace=True) #the function update the dataframe inplace.
    
    # return groups.apply(lambda x: x) # to return the DataFrameGroupBy in a DataFrame object.
        

#### Dropping rows with 0-'Paid-Off' consecutive delinquencies respect to the dates each other

In [42]:
red_data = all_data.copy()
drop_invalid_delinquency_status(red_data, invalid_consecutive_delinquencies, status=0)
display(red_data.shape)

(3831833, 196)

In [43]:
red_labels = red_data['MBA_DELINQUENCY_STATUS_next']
M = markov_tmatrix_per_loan(red_data, red_labels)
display('Transition Matrix per Loan without consecutive Paid-Off status')
display(M)
display('Probabilities Matrix per Loan without consecutive Paid-Off status')
probabilities_matrix(M)
for row in M: print(' '.join('{0:.3f}'.format(x) for x in row))  

'Transition Matrix per Loan without consecutive Paid-Off status'

[[2039, 1, 0, 10, 8, 13, 6],
 [641, 33522, 16242, 121, 26073, 14, 2],
 [164, 3745, 9317, 10453, 2559, 802, 1],
 [1381, 788, 1415, 51128, 2705, 8122, 336],
 [34388, 38554, 214, 66, 3443542, 21, 8],
 [1373, 112, 64, 4103, 896, 58503, 1777],
 [413, 3, 0, 474, 11, 80, 7993]]

'Probabilities Matrix per Loan without consecutive Paid-Off status'

0.982 0.000 0.000 0.005 0.004 0.006 0.003
0.008 0.438 0.212 0.002 0.340 0.000 0.000
0.006 0.138 0.345 0.387 0.095 0.030 0.000
0.021 0.012 0.021 0.776 0.041 0.123 0.005
0.010 0.011 0.000 0.000 0.979 0.000 0.000
0.021 0.002 0.001 0.061 0.013 0.875 0.027
0.046 0.000 0.000 0.053 0.001 0.009 0.891


After the dropping task, the dataset already contains some 'Paid-Off' consecutive statuses which are distant in months. 

From the paper (Sirignano et. al, 2017): 

"REO and paid-oﬀ are treated as absorbing states. That is, we stop tracking the mortgage after the ﬁrst time it enters REO or paid off" ... 

According to this observation, should these states be eliminated??

For example:

In [44]:
groups = red_data[['LOAN_ID', 'MBA_DELINQUENCY_STATUS_next', 'ASOFMONTH']][red_data['MBA_DELINQUENCY_STATUS_next']==0].groupby(['LOAN_ID'])
display(groups.get_group(1152166545))

,LOAN_ID,ASOFMONTH
129271,1152166545,2009-03-01
3917227,1152166545,2010-06-01


## Dropping invalid transitions between delinquency statuses:

There are other invalid transitions that are present in our current transition matrix: 
                    - current --> 60,
                    - current -->90,
                    - 30 --> 90,
                    - REO --> any other state?? (except itself??),
                    - PAID-OFF --> any other state?? (except itself??)

From Paper (Sirignano et. al, 2017): 

"Monthly mortgage transitions from current to 60 days delinquent or from 30 days delinquent to 90+ days delinquent 
are not possible. Errors of this type are very infrequent in the dataset and we remove those samples 
where such errors occur."
 
"Certain transitions are not allowed in the dataset (e.g., current to 60 days delinquent).  Although such a transition is theoretically allowed in the formulation (2), the transition probabilities of transitions which do not occur in the dataset will be driven to zero during training." 

Based on this, Should REO --> REO and PAID-OFF --> PAID-OFF be considered as invalid transitions?. After training our model, Is it necessary change the predicted output for invalid transitions in a hard-coded way???.


Tables 8, 9 and 10, extracted from Paper (Sirignano et. al, 2017):
<img src="figures_mtm/monthly_transition_matrix_from_paper.png">

Image extracted from RiskModel_V4_Technical_Document.pdf (CoreLogic Group):

<img src="figures_mtm/all_posible_mortgage_transitions.png">

It can be observed differences between the allowed states by each approach (Sirignano et. al and CoreLogic-Group). Our dataset looks more like Sirignano et. al approach. However, it is important to study these differences to build the most realistic approach for mortgages.

The following function finds the invalid transitions per loan considered above: 

In [18]:
def invalid_transition (group):
    '''
    Assuming integer labels according with the following dictionary: 
    {'0': 0, '3': 1, '6': 2, '9': 3, 'C': 4, 'F': 5, 'R': 6}
 
    '''
    uw = []
    for i in range(0, (len(group)-1)):        
        if (((group.iloc[i]['MBA_DELINQUENCY_STATUS_next'] == 4) and (group.iloc[i+1]['MBA_DELINQUENCY_STATUS_next'] == 2))             
           or ((group.iloc[i]['MBA_DELINQUENCY_STATUS_next'] == 4) and (group.iloc[i+1]['MBA_DELINQUENCY_STATUS_next'] == 3))
           or ((group.iloc[i]['MBA_DELINQUENCY_STATUS_next'] == 1) and (group.iloc[i+1]['MBA_DELINQUENCY_STATUS_next'] == 3))
           or ((group.iloc[i]['MBA_DELINQUENCY_STATUS_next'] == 0) and (group.iloc[i+1]['MBA_DELINQUENCY_STATUS_next'] != 0))
           or ((group.iloc[i]['MBA_DELINQUENCY_STATUS_next'] == 6) and (group.iloc[i+1]['MBA_DELINQUENCY_STATUS_next'] != 6))): 
            uw.append(i+1)
    return group.index[uw]

For this pruning, it will be used the dataset without 0-'Paid-Off' consecutive status as aforementioned.

In [19]:
drop_invalid_delinquency_status(red_data, invalid_transition, status=None)

After this, the dataset was reduced in 1420 records.  

In [21]:
display(red_data.shape)
red_data.to_hdf(os.path.join(pdata_dir, 'reduced_data.h5'), 'table', mode='w')

(3830413, 196)

,LOAN_ID,ASOFMONTH,MBA_DAYS_DELINQUENT,CURRENT_INTEREST_RATE,LLMA2_CURRENT_INTEREST_SPREAD,LOANAGE,CURRENT_BALANCE,SCHEDULED_PRINCIPAL,SCHEDULED_MONTHLY_PANDI,LLMA2_HIST_LAST_12_MONTHS_MIS,LLMA2_C_IN_LAST_12_MONTHS,LLMA2_30_IN_LAST_12_MONTHS,LLMA2_60_IN_LAST_12_MONTHS,LLMA2_90_IN_LAST_12_MONTHS,LLMA2_FC_IN_LAST_12_MONTHS,LLMA2_REO_IN_LAST_12_MONTHS,YEAR,MONTH,TIME,CURRENT_INTEREST_RATE_nan,MBA_DAYS_DELINQUENT_nan,SCHEDULED_MONTHLY_PANDI_nan,SCHEDULED_PRINCIPAL_nan,MBA_DELINQUENCY_STATUS_0,MBA_DELINQUENCY_STATUS_3,MBA_DELINQUENCY_STATUS_6,MBA_DELINQUENCY_STATUS_9,MBA_DELINQUENCY_STATUS_C,MBA_DELINQUENCY_STATUS_F,MBA_DELINQUENCY_STATUS_R,CURRENT_INVESTOR_CODE_253,MBA_DELINQUENCY_STATUS,FICO_SCORE_ORIGINATION,INITIAL_INTEREST_RATE,ORIGINAL_LTV,ORIGINAL_BALANCE,BACKEND_RATIO,SALE_PRICE,NUMBER_OF_UNITS,LLMA2_APPVAL_LT_SALEPRICE,LLMA2_ORIG_RATE_SPREAD,MARGIN,PERIODIC_RATE_CAP,PERIODIC_RATE_FLOOR,LIFETIME_RATE_CAP,LIFETIME_RATE_FLOOR,RATE_RESET_FREQUENCY,PAY_RESET_FREQUENCY,FIRST_RATE_RESET_PERIOD,LLMA2_PRIME,LLMA2_SUBPRIME,BACKEND_RATIO_nan,FIRST_RATE_RESET_PERIOD_nan,LIFETIME_RATE_CAP_nan,LIFETIME_RATE_FLOOR_nan,MARGIN_nan,PAY_RESET_FREQUENCY_nan,PERIODIC_RATE_CAP_nan,PERIODIC_RATE_FLOOR_nan,RATE_RESET_FREQUENCY_nan,SALE_PRICE_nan,ORIGINAL_TERM_180,ORIGINAL_TERM_240,ORIGINAL_TERM_360,LLMA2_VINTAGE_2005,BUYDOWN_FLAG_N,BUYDOWN_FLAG_U,BUYDOWN_FLAG_Y,NEGATIVE_AMORTIZATION_FLAG_N,NEGATIVE_AMORTIZATION_FLAG_U,NEGATIVE_AMORTIZATION_FLAG_Y,PREPAY_PENALTY_FLAG_N,PREPAY_PENALTY_FLAG_U,PREPAY_PENALTY_FLAG_Y,OCCUPANCY_TYPE_1,OCCUPANCY_TYPE_2,OCCUPANCY_TYPE_3,OCCUPANCY_TYPE_U,PRODUCT_TYPE_10,PRODUCT_TYPE_20,PRODUCT_TYPE_30,PRODUCT_TYPE_50,PRODUCT_TYPE_51,PRODUCT_TYPE_52,PRODUCT_TYPE_54,PRODUCT_TYPE_5A,PRODUCT_TYPE_5Z,PRODUCT_TYPE_60,PRODUCT_TYPE_63,PRODUCT_TYPE_70,PRODUCT_TYPE_80,PRODUCT_TYPE_81,PRODUCT_TYPE_82,PRODUCT_TYPE_83,PRODUCT_TYPE_84,PRODUCT_TYPE_8Z,PRODUCT_TYPE_U,LOAN_PURPOSE_1,LOAN_PURPOSE_2,LOAN_PURPOSE_3,LOAN_PURPOSE_5,LOAN_PURPOSE_6,LOAN_PURPOSE_7,LOAN_PURPOSE_8,LOAN_PURPOSE_9,LOAN_PURPOSE_B,LOAN_PURPOSE_U,DOCUMENTATION_TYPE_1,DOCUMENTATION_TYPE_2,DOCUMENTATION_TYPE_3,DOCUMENTATION_TYPE_U,CHANNEL_1,CHANNEL_2,CHANNEL_3,CHANNEL_4,CHANNEL_7,CHANNEL_8,CHANNEL_9,CHANNEL_A,CHANNEL_C,CHANNEL_D,CHANNEL_U,LOAN_TYPE_1,LOAN_TYPE_2,LOAN_TYPE_3,LOAN_TYPE_4,LOAN_TYPE_U,IO_FLAG_N,IO_FLAG_U,IO_FLAG_Y,CONVERTIBLE_FLAG_N,CONVERTIBLE_FLAG_U,CONVERTIBLE_FLAG_Y,POOL_INSURANCE_FLAG_N,POOL_INSURANCE_FLAG_U,POOL_INSURANCE_FLAG_Y,STATE_AE,STATE_AK,STATE_AL,STATE_AP,STATE_AR,STATE_AZ,STATE_CA,STATE_CO,STATE_CT,STATE_DC,STATE_DE,STATE_FL,STATE_GA,STATE_GU,STATE_HI,STATE_IA,STATE_ID,STATE_IL,STATE_IN,STATE_KS,STATE_KY,STATE_LA,STATE_MA,STATE_MD,STATE_ME,STATE_MI,STATE_MN,STATE_MO,STATE_MS,STATE_MT,STATE_NC,STATE_ND,STATE_NE,STATE_NH,STATE_NJ,STATE_NM,STATE_NN,STATE_NV,STATE_NY,STATE_OH,STATE_OK,STATE_OR,STATE_PA,STATE_PR,STATE_RI,STATE_SC,STATE_SD,STATE_TN,STATE_TX,STATE_UT,STATE_VA,STATE_VI,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY,STATE,UR,MBA_DELINQUENCY_STATUS_next
3548663,131175200,2005-10-01,0,8.375,2.305,0,67777.656250,45.950001,518.969971,0,2,0,3,6,1,0,2005,10,429,0,0,0,0,0,0,0,0,1,0,0,1,C,591.0,8.375,88.099998,68279.0,0.0,0.0,1,0.0,2.555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NH,3.6,1
3549190,131175200,2005-11-01,30,8.375,2.045,1,68279.289062,42.450001,518.969971,0,2,1,3,5,1,0,2005,11,430,0,0,0,0,0,1,0,0,0,0,0,1,3,591.0,8.375,88.099998,68279.0,0.0,0.0,1,0.0,2.555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
# to recover and check the results:
# new_data = read_hdf('storage.h5','d1',where=['A>.5'], columns=['A','B'])
new_data = pd.read_hdf(os.path.join(pdata_dir, 'reduced_data.h5'), 'table')
display(new_data)

,LOAN_ID,ASOFMONTH,MBA_DAYS_DELINQUENT,CURRENT_INTEREST_RATE,LLMA2_CURRENT_INTEREST_SPREAD,LOANAGE,CURRENT_BALANCE,SCHEDULED_PRINCIPAL,SCHEDULED_MONTHLY_PANDI,LLMA2_HIST_LAST_12_MONTHS_MIS,LLMA2_C_IN_LAST_12_MONTHS,LLMA2_30_IN_LAST_12_MONTHS,LLMA2_60_IN_LAST_12_MONTHS,LLMA2_90_IN_LAST_12_MONTHS,LLMA2_FC_IN_LAST_12_MONTHS,LLMA2_REO_IN_LAST_12_MONTHS,YEAR,MONTH,TIME,CURRENT_INTEREST_RATE_nan,MBA_DAYS_DELINQUENT_nan,SCHEDULED_MONTHLY_PANDI_nan,SCHEDULED_PRINCIPAL_nan,MBA_DELINQUENCY_STATUS_0,MBA_DELINQUENCY_STATUS_3,MBA_DELINQUENCY_STATUS_6,MBA_DELINQUENCY_STATUS_9,MBA_DELINQUENCY_STATUS_C,MBA_DELINQUENCY_STATUS_F,MBA_DELINQUENCY_STATUS_R,CURRENT_INVESTOR_CODE_253,MBA_DELINQUENCY_STATUS,FICO_SCORE_ORIGINATION,INITIAL_INTEREST_RATE,ORIGINAL_LTV,ORIGINAL_BALANCE,BACKEND_RATIO,SALE_PRICE,NUMBER_OF_UNITS,LLMA2_APPVAL_LT_SALEPRICE,LLMA2_ORIG_RATE_SPREAD,MARGIN,PERIODIC_RATE_CAP,PERIODIC_RATE_FLOOR,LIFETIME_RATE_CAP,LIFETIME_RATE_FLOOR,RATE_RESET_FREQUENCY,PAY_RESET_FREQUENCY,FIRST_RATE_RESET_PERIOD,LLMA2_PRIME,LLMA2_SUBPRIME,BACKEND_RATIO_nan,FIRST_RATE_RESET_PERIOD_nan,LIFETIME_RATE_CAP_nan,LIFETIME_RATE_FLOOR_nan,MARGIN_nan,PAY_RESET_FREQUENCY_nan,PERIODIC_RATE_CAP_nan,PERIODIC_RATE_FLOOR_nan,RATE_RESET_FREQUENCY_nan,SALE_PRICE_nan,ORIGINAL_TERM_180,ORIGINAL_TERM_240,ORIGINAL_TERM_360,LLMA2_VINTAGE_2005,BUYDOWN_FLAG_N,BUYDOWN_FLAG_U,BUYDOWN_FLAG_Y,NEGATIVE_AMORTIZATION_FLAG_N,NEGATIVE_AMORTIZATION_FLAG_U,NEGATIVE_AMORTIZATION_FLAG_Y,PREPAY_PENALTY_FLAG_N,PREPAY_PENALTY_FLAG_U,PREPAY_PENALTY_FLAG_Y,OCCUPANCY_TYPE_1,OCCUPANCY_TYPE_2,OCCUPANCY_TYPE_3,OCCUPANCY_TYPE_U,PRODUCT_TYPE_10,PRODUCT_TYPE_20,PRODUCT_TYPE_30,PRODUCT_TYPE_50,PRODUCT_TYPE_51,PRODUCT_TYPE_52,PRODUCT_TYPE_54,PRODUCT_TYPE_5A,PRODUCT_TYPE_5Z,PRODUCT_TYPE_60,PRODUCT_TYPE_63,PRODUCT_TYPE_70,PRODUCT_TYPE_80,PRODUCT_TYPE_81,PRODUCT_TYPE_82,PRODUCT_TYPE_83,PRODUCT_TYPE_84,PRODUCT_TYPE_8Z,PRODUCT_TYPE_U,LOAN_PURPOSE_1,LOAN_PURPOSE_2,LOAN_PURPOSE_3,LOAN_PURPOSE_5,LOAN_PURPOSE_6,LOAN_PURPOSE_7,LOAN_PURPOSE_8,LOAN_PURPOSE_9,LOAN_PURPOSE_B,LOAN_PURPOSE_U,DOCUMENTATION_TYPE_1,DOCUMENTATION_TYPE_2,DOCUMENTATION_TYPE_3,DOCUMENTATION_TYPE_U,CHANNEL_1,CHANNEL_2,CHANNEL_3,CHANNEL_4,CHANNEL_7,CHANNEL_8,CHANNEL_9,CHANNEL_A,CHANNEL_C,CHANNEL_D,CHANNEL_U,LOAN_TYPE_1,LOAN_TYPE_2,LOAN_TYPE_3,LOAN_TYPE_4,LOAN_TYPE_U,IO_FLAG_N,IO_FLAG_U,IO_FLAG_Y,CONVERTIBLE_FLAG_N,CONVERTIBLE_FLAG_U,CONVERTIBLE_FLAG_Y,POOL_INSURANCE_FLAG_N,POOL_INSURANCE_FLAG_U,POOL_INSURANCE_FLAG_Y,STATE_AE,STATE_AK,STATE_AL,STATE_AP,STATE_AR,STATE_AZ,STATE_CA,STATE_CO,STATE_CT,STATE_DC,STATE_DE,STATE_FL,STATE_GA,STATE_GU,STATE_HI,STATE_IA,STATE_ID,STATE_IL,STATE_IN,STATE_KS,STATE_KY,STATE_LA,STATE_MA,STATE_MD,STATE_ME,STATE_MI,STATE_MN,STATE_MO,STATE_MS,STATE_MT,STATE_NC,STATE_ND,STATE_NE,STATE_NH,STATE_NJ,STATE_NM,STATE_NN,STATE_NV,STATE_NY,STATE_OH,STATE_OK,STATE_OR,STATE_PA,STATE_PR,STATE_RI,STATE_SC,STATE_SD,STATE_TN,STATE_TX,STATE_UT,STATE_VA,STATE_VI,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY,STATE,UR,MBA_DELINQUENCY_STATUS_next
3548663,131175200,2005-10-01,0,8.375,2.305,0,67777.656250,45.950001,518.969971,0,2,0,3,6,1,0,2005,10,429,0,0,0,0,0,0,0,0,1,0,0,1,C,591.0,8.375,88.099998,68279.0,0.0,0.0,1,0.0,2.555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NH,3.6,1
3549190,131175200,2005-11-01,30,8.375,2.045,1,68279.289062,42.450001,518.969971,0,2,1,3,5,1,0,2005,11,430,0,0,0,0,0,1,0,0,0,0,0,1,3,591.0,8.375,88.099998,68279.0,0.0,0.0,1,0.0,2.555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Calculating the Transition Matrix.

In [23]:
red_labels = red_data['MBA_DELINQUENCY_STATUS_next']
M = markov_tmatrix_per_loan(red_data, red_labels)
display('Transition Matrix per Loan without invalid transitions and consecutive Paid-Off status')
display(M)
display('Probabilities Matrix per Loan without invalid transitions and consecutive Paid-Off status')
probabilities_matrix(M)
for row in M: print(' '.join('{0:.3f}'.format(x) for x in row))  

'Transition Matrix per Loan without invalid transitions and consecutive Paid-Off status'

[[2036, 1, 0, 4, 7, 10, 4],
 [641, 33526, 16258, 65, 26086, 33, 2],
 [161, 3702, 9265, 10394, 2512, 797, 1],
 [1226, 780, 1398, 50726, 2668, 8081, 335],
 [34391, 38598, 53, 86, 3443594, 39, 8],
 [1370, 112, 64, 4098, 896, 58423, 1772],
 [161, 2, 0, 311, 12, 79, 7995]]

'Probabilities Matrix per Loan without invalid transitions and consecutive Paid-Off status'

0.987 0.000 0.000 0.002 0.003 0.005 0.002
0.008 0.438 0.212 0.001 0.340 0.000 0.000
0.006 0.138 0.345 0.387 0.094 0.030 0.000
0.019 0.012 0.021 0.778 0.041 0.124 0.005
0.010 0.011 0.000 0.000 0.979 0.000 0.000
0.021 0.002 0.001 0.061 0.013 0.875 0.027
0.019 0.000 0.000 0.036 0.001 0.009 0.934


## Transition Matrix for subprime/prime datasets.

In [29]:
display(red_data[red_data['LLMA2_SUBPRIME']==1])

,LOAN_ID,ASOFMONTH,MBA_DAYS_DELINQUENT,CURRENT_INTEREST_RATE,LLMA2_CURRENT_INTEREST_SPREAD,LOANAGE,CURRENT_BALANCE,SCHEDULED_PRINCIPAL,SCHEDULED_MONTHLY_PANDI,LLMA2_HIST_LAST_12_MONTHS_MIS,LLMA2_C_IN_LAST_12_MONTHS,LLMA2_30_IN_LAST_12_MONTHS,LLMA2_60_IN_LAST_12_MONTHS,LLMA2_90_IN_LAST_12_MONTHS,LLMA2_FC_IN_LAST_12_MONTHS,LLMA2_REO_IN_LAST_12_MONTHS,YEAR,MONTH,TIME,CURRENT_INTEREST_RATE_nan,MBA_DAYS_DELINQUENT_nan,SCHEDULED_MONTHLY_PANDI_nan,SCHEDULED_PRINCIPAL_nan,MBA_DELINQUENCY_STATUS_0,MBA_DELINQUENCY_STATUS_3,MBA_DELINQUENCY_STATUS_6,MBA_DELINQUENCY_STATUS_9,MBA_DELINQUENCY_STATUS_C,MBA_DELINQUENCY_STATUS_F,MBA_DELINQUENCY_STATUS_R,CURRENT_INVESTOR_CODE_253,MBA_DELINQUENCY_STATUS,FICO_SCORE_ORIGINATION,INITIAL_INTEREST_RATE,ORIGINAL_LTV,ORIGINAL_BALANCE,BACKEND_RATIO,SALE_PRICE,NUMBER_OF_UNITS,LLMA2_APPVAL_LT_SALEPRICE,LLMA2_ORIG_RATE_SPREAD,MARGIN,PERIODIC_RATE_CAP,PERIODIC_RATE_FLOOR,LIFETIME_RATE_CAP,LIFETIME_RATE_FLOOR,RATE_RESET_FREQUENCY,PAY_RESET_FREQUENCY,FIRST_RATE_RESET_PERIOD,LLMA2_PRIME,LLMA2_SUBPRIME,BACKEND_RATIO_nan,FIRST_RATE_RESET_PERIOD_nan,LIFETIME_RATE_CAP_nan,LIFETIME_RATE_FLOOR_nan,MARGIN_nan,PAY_RESET_FREQUENCY_nan,PERIODIC_RATE_CAP_nan,PERIODIC_RATE_FLOOR_nan,RATE_RESET_FREQUENCY_nan,SALE_PRICE_nan,ORIGINAL_TERM_180,ORIGINAL_TERM_240,ORIGINAL_TERM_360,LLMA2_VINTAGE_2005,BUYDOWN_FLAG_N,BUYDOWN_FLAG_U,BUYDOWN_FLAG_Y,NEGATIVE_AMORTIZATION_FLAG_N,NEGATIVE_AMORTIZATION_FLAG_U,NEGATIVE_AMORTIZATION_FLAG_Y,PREPAY_PENALTY_FLAG_N,PREPAY_PENALTY_FLAG_U,PREPAY_PENALTY_FLAG_Y,OCCUPANCY_TYPE_1,OCCUPANCY_TYPE_2,OCCUPANCY_TYPE_3,OCCUPANCY_TYPE_U,PRODUCT_TYPE_10,PRODUCT_TYPE_20,PRODUCT_TYPE_30,PRODUCT_TYPE_50,PRODUCT_TYPE_51,PRODUCT_TYPE_52,PRODUCT_TYPE_54,PRODUCT_TYPE_5A,PRODUCT_TYPE_5Z,PRODUCT_TYPE_60,PRODUCT_TYPE_63,PRODUCT_TYPE_70,PRODUCT_TYPE_80,PRODUCT_TYPE_81,PRODUCT_TYPE_82,PRODUCT_TYPE_83,PRODUCT_TYPE_84,PRODUCT_TYPE_8Z,PRODUCT_TYPE_U,LOAN_PURPOSE_1,LOAN_PURPOSE_2,LOAN_PURPOSE_3,LOAN_PURPOSE_5,LOAN_PURPOSE_6,LOAN_PURPOSE_7,LOAN_PURPOSE_8,LOAN_PURPOSE_9,LOAN_PURPOSE_B,LOAN_PURPOSE_U,DOCUMENTATION_TYPE_1,DOCUMENTATION_TYPE_2,DOCUMENTATION_TYPE_3,DOCUMENTATION_TYPE_U,CHANNEL_1,CHANNEL_2,CHANNEL_3,CHANNEL_4,CHANNEL_7,CHANNEL_8,CHANNEL_9,CHANNEL_A,CHANNEL_C,CHANNEL_D,CHANNEL_U,LOAN_TYPE_1,LOAN_TYPE_2,LOAN_TYPE_3,LOAN_TYPE_4,LOAN_TYPE_U,IO_FLAG_N,IO_FLAG_U,IO_FLAG_Y,CONVERTIBLE_FLAG_N,CONVERTIBLE_FLAG_U,CONVERTIBLE_FLAG_Y,POOL_INSURANCE_FLAG_N,POOL_INSURANCE_FLAG_U,POOL_INSURANCE_FLAG_Y,STATE_AE,STATE_AK,STATE_AL,STATE_AP,STATE_AR,STATE_AZ,STATE_CA,STATE_CO,STATE_CT,STATE_DC,STATE_DE,STATE_FL,STATE_GA,STATE_GU,STATE_HI,STATE_IA,STATE_ID,STATE_IL,STATE_IN,STATE_KS,STATE_KY,STATE_LA,STATE_MA,STATE_MD,STATE_ME,STATE_MI,STATE_MN,STATE_MO,STATE_MS,STATE_MT,STATE_NC,STATE_ND,STATE_NE,STATE_NH,STATE_NJ,STATE_NM,STATE_NN,STATE_NV,STATE_NY,STATE_OH,STATE_OK,STATE_OR,STATE_PA,STATE_PR,STATE_RI,STATE_SC,STATE_SD,STATE_TN,STATE_TX,STATE_UT,STATE_VA,STATE_VI,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY,STATE,UR,MBA_DELINQUENCY_STATUS_next
3548663,131175200,2005-10-01,0,8.375,2.305,0,67777.656250,45.950001,518.969971,0,2,0,3,6,1,0,2005,10,429,0,0,0,0,0,0,0,0,1,0,0,1,C,591.0,8.375,88.099998,68279.0,0.0,0.0,1,0.0,2.555,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NH,3.6,1
3549190,131175200,2005-11-01,30,8.375,2.045,1,68279.289062,42.450001,518.969971,0,2,1,3,5,1,0,2005,11,430,0,0,0,0,0,1,0,0,0,0,0,1,3,591.0,8.375,88.099998,68279.0,0.0,0.0,1,0.0,2.555,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [24]:
subprime_data = red_data[red_data['LLMA2_SUBPRIME']==1]
subprime_labels = subprime_data['MBA_DELINQUENCY_STATUS_next']
M = markov_tmatrix_per_loan(subprime_data, subprime_labels)
display('Transition Matrix per SUBPRIME Loan without invalid transitions and consecutive Paid-Off status')
display(M)
display('Probabilities Matrix per SUBPRIME Loan without invalid transitions and consecutive Paid-Off status')
probabilities_matrix(M)
for row in M: print(' '.join('{0:.3f}'.format(x) for x in row))  

'Transition Matrix per SUBPRIME Loan without invalid transitions and consecutive Paid-Off status'

[[122, 1, 0, 0, 1, 1, 0],
 [140, 9942, 3820, 19, 6265, 8, 0],
 [40, 1171, 2624, 2224, 688, 121, 0],
 [132, 232, 423, 10219, 649, 1489, 39],
 [2006, 8851, 16, 26, 237051, 11, 3],
 [281, 28, 15, 739, 248, 9286, 233],
 [17, 0, 0, 34, 5, 18, 1554]]

'Probabilities Matrix per SUBPRIME Loan without invalid transitions and consecutive Paid-Off status'

0.976 0.008 0.000 0.000 0.008 0.008 0.000
0.007 0.492 0.189 0.001 0.310 0.000 0.000
0.006 0.171 0.382 0.324 0.100 0.018 0.000
0.010 0.018 0.032 0.775 0.049 0.113 0.003
0.008 0.036 0.000 0.000 0.956 0.000 0.000
0.026 0.003 0.001 0.068 0.023 0.857 0.022
0.010 0.000 0.000 0.021 0.003 0.011 0.955


In [32]:
display(red_data[red_data['LLMA2_PRIME']==1])

,LOAN_ID,ASOFMONTH,MBA_DAYS_DELINQUENT,CURRENT_INTEREST_RATE,LLMA2_CURRENT_INTEREST_SPREAD,LOANAGE,CURRENT_BALANCE,SCHEDULED_PRINCIPAL,SCHEDULED_MONTHLY_PANDI,LLMA2_HIST_LAST_12_MONTHS_MIS,LLMA2_C_IN_LAST_12_MONTHS,LLMA2_30_IN_LAST_12_MONTHS,LLMA2_60_IN_LAST_12_MONTHS,LLMA2_90_IN_LAST_12_MONTHS,LLMA2_FC_IN_LAST_12_MONTHS,LLMA2_REO_IN_LAST_12_MONTHS,YEAR,MONTH,TIME,CURRENT_INTEREST_RATE_nan,MBA_DAYS_DELINQUENT_nan,SCHEDULED_MONTHLY_PANDI_nan,SCHEDULED_PRINCIPAL_nan,MBA_DELINQUENCY_STATUS_0,MBA_DELINQUENCY_STATUS_3,MBA_DELINQUENCY_STATUS_6,MBA_DELINQUENCY_STATUS_9,MBA_DELINQUENCY_STATUS_C,MBA_DELINQUENCY_STATUS_F,MBA_DELINQUENCY_STATUS_R,CURRENT_INVESTOR_CODE_253,MBA_DELINQUENCY_STATUS,FICO_SCORE_ORIGINATION,INITIAL_INTEREST_RATE,ORIGINAL_LTV,ORIGINAL_BALANCE,BACKEND_RATIO,SALE_PRICE,NUMBER_OF_UNITS,LLMA2_APPVAL_LT_SALEPRICE,LLMA2_ORIG_RATE_SPREAD,MARGIN,PERIODIC_RATE_CAP,PERIODIC_RATE_FLOOR,LIFETIME_RATE_CAP,LIFETIME_RATE_FLOOR,RATE_RESET_FREQUENCY,PAY_RESET_FREQUENCY,FIRST_RATE_RESET_PERIOD,LLMA2_PRIME,LLMA2_SUBPRIME,BACKEND_RATIO_nan,FIRST_RATE_RESET_PERIOD_nan,LIFETIME_RATE_CAP_nan,LIFETIME_RATE_FLOOR_nan,MARGIN_nan,PAY_RESET_FREQUENCY_nan,PERIODIC_RATE_CAP_nan,PERIODIC_RATE_FLOOR_nan,RATE_RESET_FREQUENCY_nan,SALE_PRICE_nan,ORIGINAL_TERM_180,ORIGINAL_TERM_240,ORIGINAL_TERM_360,LLMA2_VINTAGE_2005,BUYDOWN_FLAG_N,BUYDOWN_FLAG_U,BUYDOWN_FLAG_Y,NEGATIVE_AMORTIZATION_FLAG_N,NEGATIVE_AMORTIZATION_FLAG_U,NEGATIVE_AMORTIZATION_FLAG_Y,PREPAY_PENALTY_FLAG_N,PREPAY_PENALTY_FLAG_U,PREPAY_PENALTY_FLAG_Y,OCCUPANCY_TYPE_1,OCCUPANCY_TYPE_2,OCCUPANCY_TYPE_3,OCCUPANCY_TYPE_U,PRODUCT_TYPE_10,PRODUCT_TYPE_20,PRODUCT_TYPE_30,PRODUCT_TYPE_50,PRODUCT_TYPE_51,PRODUCT_TYPE_52,PRODUCT_TYPE_54,PRODUCT_TYPE_5A,PRODUCT_TYPE_5Z,PRODUCT_TYPE_60,PRODUCT_TYPE_63,PRODUCT_TYPE_70,PRODUCT_TYPE_80,PRODUCT_TYPE_81,PRODUCT_TYPE_82,PRODUCT_TYPE_83,PRODUCT_TYPE_84,PRODUCT_TYPE_8Z,PRODUCT_TYPE_U,LOAN_PURPOSE_1,LOAN_PURPOSE_2,LOAN_PURPOSE_3,LOAN_PURPOSE_5,LOAN_PURPOSE_6,LOAN_PURPOSE_7,LOAN_PURPOSE_8,LOAN_PURPOSE_9,LOAN_PURPOSE_B,LOAN_PURPOSE_U,DOCUMENTATION_TYPE_1,DOCUMENTATION_TYPE_2,DOCUMENTATION_TYPE_3,DOCUMENTATION_TYPE_U,CHANNEL_1,CHANNEL_2,CHANNEL_3,CHANNEL_4,CHANNEL_7,CHANNEL_8,CHANNEL_9,CHANNEL_A,CHANNEL_C,CHANNEL_D,CHANNEL_U,LOAN_TYPE_1,LOAN_TYPE_2,LOAN_TYPE_3,LOAN_TYPE_4,LOAN_TYPE_U,IO_FLAG_N,IO_FLAG_U,IO_FLAG_Y,CONVERTIBLE_FLAG_N,CONVERTIBLE_FLAG_U,CONVERTIBLE_FLAG_Y,POOL_INSURANCE_FLAG_N,POOL_INSURANCE_FLAG_U,POOL_INSURANCE_FLAG_Y,STATE_AE,STATE_AK,STATE_AL,STATE_AP,STATE_AR,STATE_AZ,STATE_CA,STATE_CO,STATE_CT,STATE_DC,STATE_DE,STATE_FL,STATE_GA,STATE_GU,STATE_HI,STATE_IA,STATE_ID,STATE_IL,STATE_IN,STATE_KS,STATE_KY,STATE_LA,STATE_MA,STATE_MD,STATE_ME,STATE_MI,STATE_MN,STATE_MO,STATE_MS,STATE_MT,STATE_NC,STATE_ND,STATE_NE,STATE_NH,STATE_NJ,STATE_NM,STATE_NN,STATE_NV,STATE_NY,STATE_OH,STATE_OK,STATE_OR,STATE_PA,STATE_PR,STATE_RI,STATE_SC,STATE_SD,STATE_TN,STATE_TX,STATE_UT,STATE_VA,STATE_VI,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY,STATE,UR,MBA_DELINQUENCY_STATUS_next
1657429,217010733,2011-03-01,0,6.250,1.410,74,67151.031250,0.0,608.859985,1,0,0,0,0,0,0,2011,3,494,0,0,0,0,0,0,0,0,1,0,0,1,C,671.0,6.25,85.000000,83300.0,40.0,0.0,1,0.0,0.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,KY,9.6,4
1657755,217010733,2011-04-01,0,6.250,1.410,75,66891.906250,0.0,608.859985,1,0,0,0,0,0,0,2011,4,495,0,0,0,0,0,0,0,0,1,0,0,1,C,671.0,6.25,85.000000,83300.0,40.0,0.0,1,0.0,0.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,KY,9.5,4

In [25]:
prime_data = red_data[red_data['LLMA2_PRIME']==1]
prime_labels = prime_data['MBA_DELINQUENCY_STATUS_next']
M = markov_tmatrix_per_loan(prime_data, prime_labels)
display('Transition Matrix per PRIME Loan without invalid transitions and consecutive Paid-Off status')
display(M)
display('Probabilities Matrix per PRIME Loan without invalid transitions and consecutive Paid-Off status')
probabilities_matrix(M)
for row in M: print(' '.join('{0:.3f}'.format(x) for x in row))

'Transition Matrix per PRIME Loan without invalid transitions and consecutive Paid-Off status'

[[1872, 0, 0, 4, 6, 9, 4],
 [496, 23357, 12283, 46, 19655, 25, 2],
 [121, 2505, 6574, 8058, 1810, 665, 1],
 [1068, 542, 960, 39900, 1992, 6507, 293],
 [31975, 29457, 37, 60, 3173828, 28, 5],
 [1075, 84, 49, 3313, 644, 48599, 1521],
 [139, 2, 0, 269, 7, 61, 6333]]

'Probabilities Matrix per PRIME Loan without invalid transitions and consecutive Paid-Off status'

0.988 0.000 0.000 0.002 0.003 0.005 0.002
0.009 0.418 0.220 0.001 0.352 0.000 0.000
0.006 0.127 0.333 0.408 0.092 0.034 0.000
0.021 0.011 0.019 0.778 0.039 0.127 0.006
0.010 0.009 0.000 0.000 0.981 0.000 0.000
0.019 0.002 0.001 0.060 0.012 0.879 0.028
0.020 0.000 0.000 0.039 0.001 0.009 0.930


The transition matrices for Prime, SubPrime and the full dataset looks quite similiar to the results shown above by Sirignano et. al, 2017 at tables 8, 9 and 10.

## Loans that do not belong to Prime/Subprime types

What kind of loan the following records belongs??

In [26]:
u_data =red_data[(red_data['LLMA2_PRIME']==0) & (red_data['LLMA2_SUBPRIME']==0)]
display(u_data)

,LOAN_ID,ASOFMONTH,MBA_DAYS_DELINQUENT,CURRENT_INTEREST_RATE,LLMA2_CURRENT_INTEREST_SPREAD,LOANAGE,CURRENT_BALANCE,SCHEDULED_PRINCIPAL,SCHEDULED_MONTHLY_PANDI,LLMA2_HIST_LAST_12_MONTHS_MIS,LLMA2_C_IN_LAST_12_MONTHS,LLMA2_30_IN_LAST_12_MONTHS,LLMA2_60_IN_LAST_12_MONTHS,LLMA2_90_IN_LAST_12_MONTHS,LLMA2_FC_IN_LAST_12_MONTHS,LLMA2_REO_IN_LAST_12_MONTHS,YEAR,MONTH,TIME,CURRENT_INTEREST_RATE_nan,MBA_DAYS_DELINQUENT_nan,SCHEDULED_MONTHLY_PANDI_nan,SCHEDULED_PRINCIPAL_nan,MBA_DELINQUENCY_STATUS_0,MBA_DELINQUENCY_STATUS_3,MBA_DELINQUENCY_STATUS_6,MBA_DELINQUENCY_STATUS_9,MBA_DELINQUENCY_STATUS_C,MBA_DELINQUENCY_STATUS_F,MBA_DELINQUENCY_STATUS_R,CURRENT_INVESTOR_CODE_253,MBA_DELINQUENCY_STATUS,FICO_SCORE_ORIGINATION,INITIAL_INTEREST_RATE,ORIGINAL_LTV,ORIGINAL_BALANCE,BACKEND_RATIO,SALE_PRICE,NUMBER_OF_UNITS,LLMA2_APPVAL_LT_SALEPRICE,LLMA2_ORIG_RATE_SPREAD,MARGIN,PERIODIC_RATE_CAP,PERIODIC_RATE_FLOOR,LIFETIME_RATE_CAP,LIFETIME_RATE_FLOOR,RATE_RESET_FREQUENCY,PAY_RESET_FREQUENCY,FIRST_RATE_RESET_PERIOD,LLMA2_PRIME,LLMA2_SUBPRIME,BACKEND_RATIO_nan,FIRST_RATE_RESET_PERIOD_nan,LIFETIME_RATE_CAP_nan,LIFETIME_RATE_FLOOR_nan,MARGIN_nan,PAY_RESET_FREQUENCY_nan,PERIODIC_RATE_CAP_nan,PERIODIC_RATE_FLOOR_nan,RATE_RESET_FREQUENCY_nan,SALE_PRICE_nan,ORIGINAL_TERM_180,ORIGINAL_TERM_240,ORIGINAL_TERM_360,LLMA2_VINTAGE_2005,BUYDOWN_FLAG_N,BUYDOWN_FLAG_U,BUYDOWN_FLAG_Y,NEGATIVE_AMORTIZATION_FLAG_N,NEGATIVE_AMORTIZATION_FLAG_U,NEGATIVE_AMORTIZATION_FLAG_Y,PREPAY_PENALTY_FLAG_N,PREPAY_PENALTY_FLAG_U,PREPAY_PENALTY_FLAG_Y,OCCUPANCY_TYPE_1,OCCUPANCY_TYPE_2,OCCUPANCY_TYPE_3,OCCUPANCY_TYPE_U,PRODUCT_TYPE_10,PRODUCT_TYPE_20,PRODUCT_TYPE_30,PRODUCT_TYPE_50,PRODUCT_TYPE_51,PRODUCT_TYPE_52,PRODUCT_TYPE_54,PRODUCT_TYPE_5A,PRODUCT_TYPE_5Z,PRODUCT_TYPE_60,PRODUCT_TYPE_63,PRODUCT_TYPE_70,PRODUCT_TYPE_80,PRODUCT_TYPE_81,PRODUCT_TYPE_82,PRODUCT_TYPE_83,PRODUCT_TYPE_84,PRODUCT_TYPE_8Z,PRODUCT_TYPE_U,LOAN_PURPOSE_1,LOAN_PURPOSE_2,LOAN_PURPOSE_3,LOAN_PURPOSE_5,LOAN_PURPOSE_6,LOAN_PURPOSE_7,LOAN_PURPOSE_8,LOAN_PURPOSE_9,LOAN_PURPOSE_B,LOAN_PURPOSE_U,DOCUMENTATION_TYPE_1,DOCUMENTATION_TYPE_2,DOCUMENTATION_TYPE_3,DOCUMENTATION_TYPE_U,CHANNEL_1,CHANNEL_2,CHANNEL_3,CHANNEL_4,CHANNEL_7,CHANNEL_8,CHANNEL_9,CHANNEL_A,CHANNEL_C,CHANNEL_D,CHANNEL_U,LOAN_TYPE_1,LOAN_TYPE_2,LOAN_TYPE_3,LOAN_TYPE_4,LOAN_TYPE_U,IO_FLAG_N,IO_FLAG_U,IO_FLAG_Y,CONVERTIBLE_FLAG_N,CONVERTIBLE_FLAG_U,CONVERTIBLE_FLAG_Y,POOL_INSURANCE_FLAG_N,POOL_INSURANCE_FLAG_U,POOL_INSURANCE_FLAG_Y,STATE_AE,STATE_AK,STATE_AL,STATE_AP,STATE_AR,STATE_AZ,STATE_CA,STATE_CO,STATE_CT,STATE_DC,STATE_DE,STATE_FL,STATE_GA,STATE_GU,STATE_HI,STATE_IA,STATE_ID,STATE_IL,STATE_IN,STATE_KS,STATE_KY,STATE_LA,STATE_MA,STATE_MD,STATE_ME,STATE_MI,STATE_MN,STATE_MO,STATE_MS,STATE_MT,STATE_NC,STATE_ND,STATE_NE,STATE_NH,STATE_NJ,STATE_NM,STATE_NN,STATE_NV,STATE_NY,STATE_OH,STATE_OK,STATE_OR,STATE_PA,STATE_PR,STATE_RI,STATE_SC,STATE_SD,STATE_TN,STATE_TX,STATE_UT,STATE_VA,STATE_VI,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY,STATE,UR,MBA_DELINQUENCY_STATUS_next
2556733,217237944,2005-05-01,0,6.500,0.780,1,463158.78125,0.000000,2932.800049,1,0,0,0,0,0,0,2005,5,424,0,0,0,1,0,0,0,0,1,0,0,1,C,649.0,6.500,80.000000,464000.0,0.000000,0.000000e+00,1,0.0,0.870,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NY,4.9,4
2556285,217237944,2005-06-01,0,6.500,0.920,2,462734.78125,0.000000,2932.800049,1,0,0,0,0,0,0,2005,6,425,0,0,0,1,0,0,0,0,1,0,0,1,C,649.0,6.500,80.000000,464000.0,0.000000,0.000000e+00,1,0.0,0.870,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,

In [40]:
gbu_data = u_data[['LOAN_ID', 'ASOFMONTH']].groupby('LOAN_ID').count()
gbu_data

,ASOFMONTH
LOAN_ID,
217237944,2
217644570,1
218223011,1
218232883,1
218737965,2
218848629,13
218848636,13
218848785,13
219089093,13


In [ ]:
red_data.drop(u_data.index, inplace=True)

The following instructions try to find records marked with incorrect labels inside the full dataset, but the results show a empty dataframe:

In [31]:
display(red_data.shape)
display(red_data[red_data['LOAN_ID'].isin(u_data['LOAN_ID'])][['LOAN_ID', 'ASOFMONTH', 'MBA_DELINQUENCY_STATUS_next']])

(3793779, 196)

,LOAN_ID,ASOFMONTH,MBA_DELINQUENCY_STATUS_next


## Transition Matrix for unknown type of loans

In [33]:
u_labels = u_data['MBA_DELINQUENCY_STATUS_next']
M = markov_tmatrix_per_loan(u_data, u_labels)
display('Transition Matrix per Unknown type of Loans without invalid transitions and consecutive Paid-Off status')
display(M)
display('Probabilities Matrix per Unknown type of Loans without invalid transitions and consecutive Paid-Off status')
probabilities_matrix(M)
for row in M: print(' '.join('{0:.3f}'.format(x) for x in row))

'Transition Matrix per Unknown type of Loans without invalid transitions and consecutive Paid-Off status'

[[42, 0, 0, 0, 0, 0, 0],
 [5, 227, 155, 0, 166, 0, 0],
 [0, 26, 67, 112, 14, 11, 0],
 [26, 6, 15, 607, 27, 85, 3],
 [410, 290, 0, 0, 32715, 0, 0],
 [14, 0, 0, 46, 4, 538, 18],
 [5, 0, 0, 8, 0, 0, 108]]

'Probabilities Matrix per Unknown type of Loans without invalid transitions and consecutive Paid-Off status'

1.000 0.000 0.000 0.000 0.000 0.000 0.000
0.009 0.410 0.280 0.000 0.300 0.000 0.000
0.000 0.113 0.291 0.487 0.061 0.048 0.000
0.034 0.008 0.020 0.789 0.035 0.111 0.004
0.012 0.009 0.000 0.000 0.979 0.000 0.000
0.023 0.000 0.000 0.074 0.006 0.868 0.029
0.041 0.000 0.000 0.066 0.000 0.000 0.893


How these type of loans should be treated?

In [41]:
red_data.shape

(3793779, 196)